In [55]:
## Importing Dependencies
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [56]:
# Setup the model
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [57]:
# Calculate Sentiment
tokens = tokenizer.encode("Tasty!", return_tensors = 'pt')
result = model(tokens)
result.logits

tensor([[-1.4277, -1.2743, -0.1505,  0.9500,  1.5098]],
       grad_fn=<AddmmBackward0>)

In [58]:
int(torch.argmax(result.logits)) +1

5

In [59]:
# Collect Reviews
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [60]:
results

[<p class="comment__09f24__D0cxf css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">Some of the best Milkshakes me and my daughter ever tasted. MMMMMM HMMMMMMMM.</span></p>,
 <p class="comment__09f24__D0cxf css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so much that I apologize for not taking any photos. <br/>We all enjoyed our food, as well as our coffee and tea drinks.<br/>We were greeted immediately by a friendly server asking if we would like to sit inside or out. We said we would like inside, but weren't exactly sure how many were joining us yet- at least 4. We were told this was no problem, the more the merrier. A few minutes later when 4 more joined our party and we explained to the server we had 6, he just quickly switched our table. <br/>I really enjoyed my serenity tea, just what I needed after a long flight in from Sfo that morning. Everyone else were more intereste

In [61]:
results[0].text

'Some of the best Milkshakes me and my daughter ever tasted. MMMMMM HMMMMMMMM.'

In [62]:
# Load Review into DataFrame and Score
import pandas as pd
import numpy as np
df = pd.DataFrame(reviews, columns = ['review'])

In [63]:
df['review'].iloc[0]

'Some of the best Milkshakes me and my daughter ever tasted. MMMMMM HMMMMMMMM.'

In [64]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors = 'pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [65]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))
df

,review,sentiment
0,Some of the best Milkshakes me and my daughter...,5
1,Six of us met here for breakfast before our wa...,4
2,Great place with delicious food and friendly s...,5
3,Great food amazing coffee and tea. Short walk ...,5
4,It was ok. Had coffee with my friends. I'm new...,3
5,Ricotta hot cakes! These were so yummy. I ate ...,5
6,Great staff and food. Must try is the pan fri...,5
7,We came for brunch twice in our week-long visi...,4
8,I came to Social brew cafe for brunch while ex...,5
9,It was ok. The coffee wasn't the best but it w...,3
